### 3. Create another copy of your Python Codes, name it TTE-v2 (use Jupyter Notebook).

In [ ]:
import os
import tempfile
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

class TrialSequence:
    def __init__(self, estimand):
        self.estimand = estimand
        self.data = None
        self.switch_weight_model = None
        self.censor_weight_model = None
    
    def set_data(self, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
        self.data = data.copy()
        self.data.rename(columns={
            id_col: "id", period_col: "period", treatment_col: "treatment",
            outcome_col: "outcome", eligible_col: "eligible"
        }, inplace=True)
        return self
    
    def set_switch_weight_model(self, numerator, denominator, save_path):
        os.makedirs(save_path, exist_ok=True)
        self.switch_weight_model = {
            "numerator": numerator,
            "denominator": denominator,
            "model_fitter": "stats_glm_logit",
            "save_path": save_path
        }
        return self
    
    def set_censor_weight_model(self, censor_event, numerator, denominator, pool_models, save_path):
        os.makedirs(save_path, exist_ok=True)
        self.censor_weight_model = {
            "censor_event": censor_event,
            "numerator": numerator,
            "denominator": denominator,
            "pool_models": pool_models,
            "model_fitter": "stats_glm_logit",
            "save_path": save_path
        }
        return self

# Load data
data_censored = pd.read_csv("data_censored.csv")

# Define trials
trial_pp = TrialSequence(estimand="PP").set_data(data_censored, "id", "period", "treatment", "outcome", "eligible")
trial_itt = TrialSequence(estimand="ITT").set_data(data_censored, "id", "period", "treatment", "outcome", "eligible")

# Define paths
trial_pp_dir = os.path.join(os.getcwd(), "trial_pp")
trial_itt_dir = os.path.join(os.getcwd(), "trial_itt")

# Set models for PP
trial_pp.set_switch_weight_model("treatment ~ age", "treatment ~ age + x1 + x3", os.path.join(trial_pp_dir, "switch_models"))
trial_pp.set_censor_weight_model("censored", "1 - censored ~ x2", "1 - censored ~ x2 + x1", "none", os.path.join(trial_pp_dir, "censor_models"))

# Set models for ITT
trial_itt.set_censor_weight_model("censored", "1 - censored ~ x2", "1 - censored ~ x2 + x1", "numerator", os.path.join(trial_itt_dir, "censor_models"))

# Winsorize weights
weights = trial_itt.data["weight"] * trial_itt.data["sample_weight"]
q99 = np.quantile(weights, 0.99)
weights = np.minimum(weights, q99)

# Fit logistic regression model
X = trial_itt.data[["assigned_treatment", "x2", "followup_time"]]
X["followup_time^2"] = X["followup_time"]**2
X["trial_period"] = trial_itt.data["trial_period"]
X["trial_period^2"] = X["trial_period"]**2
X = sm.add_constant(X)
y = trial_itt.data["outcome"]
logit_model = sm.GLM(y, X, family=sm.families.Binomial(), weights=weights).fit()
print(logit_model.summary())

# Kaplan-Meier survival analysis
newdata = trial_itt.data[trial_itt.data["trial_period"] == 1]
predict_times = np.arange(0, 11)
kmf_treated = KaplanMeierFitter()
kmf_control = KaplanMeierFitter()
kmf_treated.fit(newdata[newdata["assigned_treatment"] == 1]["followup_time"], event_observed=newdata[newdata["assigned_treatment"] == 1]["outcome"], label="Treated")
kmf_control.fit(newdata[newdata["assigned_treatment"] == 0]["followup_time"], event_observed=newdata[newdata["assigned_treatment"] == 0]["outcome"], label="Control")

# Compute survival difference
surv_prob_treated = kmf_treated.predict(predict_times)
surv_prob_control = kmf_control.predict(predict_times)
survival_diff = surv_prob_treated - surv_prob_control
ci_lower, ci_upper = survival_diff - 1.96*np.std(survival_diff), survival_diff + 1.96*np.std(survival_diff)

# Plot results
plt.figure(figsize=(8, 5))
plt.plot(predict_times, survival_diff, label="Survival Difference", color="blue")
plt.fill_between(predict_times, ci_lower, ci_upper, color="red", alpha=0.2, label="95% CI")
plt.xlabel("Follow-up Time")
plt.ylabel("Survival Difference")
plt.legend()
plt.title("Survival Probability Difference Over Time")
plt.show()